In [1]:
import pandas as pd
import numpy as np
import datetime
import cftime
import xarray as xr

In [ ]:
def create_kernel():
    lat = np.arange(-89.5,-34.5,1)
    lon = np.arange(0.625, 359.875, 1.25)
    time0 = cftime.DatetimeNoLeap(151,1,1,6)
    time_delta = 0
    i0 = 0
    j0 = 0
    mslp = 90000

    output_file_static = f"/work/Katherine.Turner/tempest-extremes/kernel/kernel_list.txt"

    with open(output_file_static, 'w') as out_file:
        for i in lon:
            for j in lat:    
                tmask = time0 + datetime.timedelta(hours=int(time_delta))
                time_delta +=6
                entry_header = (int(tmask.strftime("%Y")),int(tmask.strftime("%m")),
                                int(tmask.strftime("%d")),int(tmask.strftime("%H")))
                entry_data = (i0,j0,i,j,mslp,
                              int(tmask.strftime("%Y")),int(tmask.strftime("%m")),
                              int(tmask.strftime("%d")),int(tmask.strftime("%H")))
        
                header = "start\t1" + str(entry_header).replace("(", "\t").replace(", ","\t").replace(")", "\n")
                out_file.write(header)
                data = str(entry_data).replace("(", "\t").replace(", ","\t").replace(")", "\n")
                out_file.write(data)

In [ ]:
def repeat_histogram(expname):
    """
    Creates a histogram of storm nodes (using bins consistent with the c96 atmospheric edges) and creates a 
    repeated histogram with a time dimension. This can then be used with Tempest-Extremes to create a masked
    3d histogram, which can then be summed over the area to create a smooth histogram that measures number
    of nodes within 5 degrees of a location
    """
    data = np.loadtxt(f"/work/Katherine.Turner/tempest-extremes/{expname}/node_files/SH_stormtraj.csv", delimiter="\t", skiprows=1)
    
    lat_edges = np.arange(-90,91,1)
    lon_edges = np.arange(0, 360.1, 1.25)

    lat = np.arange(-89.5,90, 1)
    lon = np.arange(.625, 360, 1.25)

    ds_kernel = xr.open_dataset("/work/Katherine.Turner/tempest-extremes/kernel/kernel_mask.nc")
    time_full = ds_kernel.time.data
    
    heatmap, xedges, yedges = np.histogram2d(data[:,6], data[:,5], bins=[lat_edges, lon_edges])
    
    heatmap_3d = np.expand_dims(heatmap, 2)
    heatmap_3d_repeat = np.repeat(heatmap_3d, 21900, axis=2)

    da = xr.DataArray(
        data = np.transpose(heatmap_3d_repeat[:,:,:15840], axes=[2,0,1]),
        dims = ["time", "lat", "lon"],
        coords = dict(
            lon = ("lon", lon),
            lat = ("lat", lat),
            time = ("time", time_full[:15840])
        ),
        attrs = dict(
            description=f"Storm density in {expname}"
        )
    )

    ds_out = da.to_dataset(name="storm_dens")
    ds_out.to_netcdf(f"/work/Katherine.Turner/tempest-extremes/kernel/histogram_{expname}_3d.nc")

In [ ]:
repeat_histogram("odiv-251")

In [14]:
def create_smooth_histogram(expname):
    ds = xr.open_dataset(f'/work/Katherine.Turner/tempest-extremes/kernel/mhist_{expname}_3d.nc', use_cftime = True)
    
    hist_out = np.reshape(ds.storm_dens.sum(dim=["lat", "lon"]).data / (95*12), (288,55)).T
    
    lat_full = np.arange(-89.5,90, 1)
    lat = lat_full[:55]
    lon = np.arange(.625, 360, 1.25)
    
    da = xr.DataArray(
        data = hist_out,
        dims = ["lat", "lon"],
        coords = dict(
            lon = ("lon", lon),
            lat = ("lat", lat)
        ),
        attrs = dict(
            description=f"5 degree density (storms per month) for {expname}"
        )
    )
    ds_out = da.to_dataset(name="hist_5deg")
    ds_out.to_netcdf(f"/work/Katherine.Turner/tempest-extremes/kernel/hist5deg_{expname}.nc")

In [15]:
create_smooth_histogram("odiv-251")